In [ ]:
# Analysis of colours dataset

##########################
######## SETTINGS ########

# jupyter matplotlib configs:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# file containing dataset of detected colours
detected_cols_file = 'detected_colours1.npy'

# Save plots or not?
save_plots = False

##########################
##########################


# imports, helper functions and load data
import numpy as np
import cv2 
from itertools import groupby
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans, MeanShift, AgglomerativeClustering, AffinityPropagation
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
from adjustText import adjust_text #https://github.com/Phlya/adjustText/wiki


# helper functions RGB <-> HSV <-> XYV

def rgb2hsv(rgb):
    return cv2.cvtColor(np.expand_dims(rgb, axis = 0), cv2.COLOR_RGB2HSV).reshape(-1,3).astype('f4')

# convert cylindrical HSV or HCV to cartesian XYV  
def cyl2xyv(cyl):
    xyv = np.zeros(cyl.shape)
    xyv[:,0] = cyl[:,1]*np.cos(cyl[:,0]/180*np.pi)
    xyv[:,1] = cyl[:,1]*np.sin(cyl[:,0]/180*np.pi)
    xyv[:,2] = cyl[:,2]
    return xyv

# convert cartesian XYV to cylindrical HSV
def xyv2cyl(xyv):
    cyl = np.zeros(xyv.shape)
    cyl[:,0] = ((np.arctan2(xyv[:,1], xyv[:,0]) + 2*np.pi) % (2 * np.pi))/np.pi*180
    cyl[:,1] = np.sqrt(xyv[:,0]**2 + xyv[:,1]**2)
    cyl[:,2] = xyv[:,2]
    return cyl.astype('f4')

def hsv2rgb(hsv):
    return cv2.cvtColor(np.expand_dims(hsv, axis = 0), cv2.COLOR_HSV2RGB).reshape(-1,3)     

def hsv2hcv(hsv):
    hcv = hsv.copy()
    hcv[:,1] = hcv[:,1]*hcv[:,2]/255
    return hcv

def hcv2hsv(hcv):
    hsv = hcv.copy()
    hsv[:,1] = hsv[:,1]/hsv[:,2]*255
    return hsv

def rgb2lab(rgb):
    return cv2.cvtColor(np.expand_dims(rgb/255, axis = 0), cv2.COLOR_RGB2Lab).reshape(-1,3)

def lab2rgb(lab):
    return cv2.cvtColor(np.expand_dims(lab, axis = 0), cv2.COLOR_Lab2RGB).reshape(-1,3)*255

# test clustering accuracy
def test_clusters(centers=None, labels=None, lower = (0,0,0), upper = (125,125,125), raw = None):
    selected_labels = []
    # determine which clusters are between the lower and upper bounds
    for i,e in enumerate(centers.tolist()):
        if ((e[0]>lower[0])&(e[0]<upper[0])&
            (e[1]>lower[1])&(e[1]<upper[1])&
            (e[2]>lower[2])&(e[2]<upper[2])): selected_labels.append(i)
    print('Lower {}, upper {}, cluster labels {}'.format(lower, upper, selected_labels))
    
    # determine how many points in the raw dataset are between the lower and upper bounds
    try:
        points_in_raw = np.sum((raw[:,0]>lower[0])&(raw[:,0]<upper[0])&
                               (raw[:,1]>lower[1])&(raw[:,1]<upper[1])&
                               (raw[:,2]>lower[2])&(raw[:,2]<upper[2]))
        print('total in raw dataset: ', points_in_raw)
    except: print("Original dataset not specified. Add raw = <dataset> when calling the function")
    # print how many points from the raw dataset belong to the clusters of interest
    print('points in cluster: {}\n'.format(len([i for i in labels if i in selected_labels])))

# convert array of labels to histogram in percentage units
def hist_pct (labels):
    hist = np.histogram(labels, bins = len(np.unique(labels)))[0].astype("float")
    hist = hist / hist.sum()*100
    return hist

# code from     https://www.pyimagesearch.com/2014/05/26/opencv-python-k-means-color-clustering/
def centroid_histogram(clt):
    # grab the number of different clusters and create a histogram based on the number of pixels assigned to each cluster
    numLabels = len(np.unique(clt.labels_))
    (hist, _) = np.histogram(clt.labels_, bins = numLabels)
    hist = hist.astype("float") # normalize the histogram, such that it sums to one
    hist /= hist.sum()
    return hist

# code from     https://www.pyimagesearch.com/2014/05/26/opencv-python-k-means-color-clustering/
def plot_colors(hist, centroids):
    # initialize the bar chart representing the relative frequency
    # of each of the colors
    bar = np.zeros((50, 300, 3), dtype = "uint8")
    startX = 0

    # loop over the percentage of each cluster and the color of each cluster
    for (percent, color) in zip(hist, centroids):
        # plot the relative percentage of each cluster
        endX = startX + (percent * 300)
        cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
        color.astype("uint8").tolist(), -1)
        startX = endX
    return bar

# Scale each axis to [-0.5, 0.5]
# if preserve_proportions == True (default), the axis with the most max-min scales to [-0.5; 0.5], 
#    and this axis' scaling factor is appplied to all other axes. 
#    All axes are centred individually (max[i]=abs(min[i]))
# if preserve_proportions == False, each axis is scaled individually to [-0.5; 0.5] 

def scale(dataset, preserve_proportions = True):
    d_max = np.max(dataset, axis = 0) 
    d_min = np.min(dataset, axis = 0)
    mean = 0.5*(d_min + d_max)
    if preserve_proportions: delta = np.max(d_max - d_min)
    else:                    delta = d_max - d_min  
    return (dataset - mean)/delta



np.set_printoptions(precision=2)

rgb_raw = np.load(detected_cols_file)

# exclude four erroneous datapoints
#rgb_raw = rgb_raw[rgb_raw[:,0]>35, :]

print('Loaded RGB array: ', rgb_raw.shape, rgb_raw.dtype)

In [ ]:
# RGB PCA

pca_model = PCA(n_components=2).fit(rgb_raw)
#print(pca_model.explained_variance_ratio_)  
#print(pca_model.components_)

In [ ]:
# RGB representation with scaling

# standardize
scaler = StandardScaler().fit(rgb_raw)
rgb = scaler.transform(rgb_raw)

#fig = plt.figure(figsize=(10,8)).add_subplot(111, projection="3d")
#fig.scatter(rgb[:,0], rgb[:,1], rgb[:,2], c=rgb_raw/255, alpha = 1, marker="o", s=20)
#fig.savefig("rgb-3d.svg", format="svg")

In [ ]:
# RGB representation

# 3d visualization 

fig = plt.figure(figsize=(10,8)).add_subplot(111, projection='3d')
#ax = plt3(fig, rect=[0, 0, .95, 1], elev=8, azim=14)

fig.scatter(rgb[:,0], rgb[:,1], rgb[:,2], 
           c=rgb_raw/255, alpha = 1, marker="o", s=35)
fig.set_xlabel('Red (z-score)')
fig.set_ylabel('Green (z-score)')
fig.set_zlabel('Blue (z-score)')

if save_plots: plt.savefig("figs/rgb-3d.svg", format="svg")

rgb_raw_pca = pca_model.transform(rgb_raw)


fig = plt.figure(figsize=(9,6))
plt.scatter(rgb_raw_pca[:,0], rgb_raw_pca[:,1], s=20, c=rgb_raw/255, alpha=1 )
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title('2D representation of all detected colours', 
          fontdict={'fontsize': 16,
                    'fontweight' : 16,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

if save_plots: plt.savefig("figs/pca-all.svg", format="svg")

In [ ]:
# create dataset for 3d web applet

#dataset = scale(rgb)

for coords, color in zip(rgb, rgb_raw):
    print("[{:.3f}, {:.3f}, {:.3f}, {:#x}],".format(coords[0], coords[1], coords[2],
                                                     int(np.dot(color, np.array([256*256, 256, 1])))))

In [ ]:
# RGB representation with scaling
# Hierarchical clustering

#('ward', 'average', 'complete', 'single'):
cl_model = AgglomerativeClustering(linkage='ward', n_clusters=30).fit(rgb)

# assign random colour to each point according to label
cols = np.random.randint(50,200, size=(cl_model.n_clusters, 3))[cl_model.labels_]/255
# plot
plt.figure(figsize=(9,6))
plt.scatter(rgb_raw_pca[:, 0], 
            rgb_raw_pca[:, 1], 
            c=cols, alpha = 1, marker="o", s=20)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Agg. clustering (Ward's method, n = {}) in RGB".format(cl_model.n_clusters), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
if save_plots: plt.savefig("figs/rgb-ac-all.svg", format="svg")


# calculate centers by averaging across clusters
centers = np.array([np.mean(rgb[cl_model.labels_==i], axis = 0) for i in range(cl_model.n_clusters)])

# destandardize
centers = scaler.inverse_transform(centers) 

centers_pca = pca_model.transform(centers)

# show cluster centers with frequency labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Agg. clustering (Ward's method, n = {}) in RGB: centres and frequencies".format(cl_model.n_clusters),  
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
labels = [plt.text(coords[0], coords[1], '{:.1f}%'.format(label), ha='center', va='center') 
         for label, coords in zip(hist_pct(cl_model.labels_), centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)
if save_plots: plt.savefig("figs/rgb-ac-freq.svg", format="svg")

# show cluster centers with index labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Agg. clustering (Ward's method) in RGB", 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
labels = [plt.text(coords[0], coords[1], '{}'.format(i), ha='center', va='center') 
         for i, coords in enumerate(centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)


test_clusters(centers, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw=rgb_raw)
test_clusters(centers, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw=rgb_raw)

In [ ]:
# RGB representation with scaling
# Hierarchial clustering
# sort rgb_ac_centers to create a sequence for the histogram plot

centers_hsv = rgb2hsv(centers)
# sort by V in HSV
v_sort_ind = np.argsort(centers_hsv[:, 2])

# grey
grey_ind = v_sort_ind[centers_hsv[v_sort_ind,1]<0.25]
# blue
blue_ind = v_sort_ind[(centers_hsv[v_sort_ind,1]>0.25) & (centers_hsv[v_sort_ind,0]>180)& (centers_hsv[v_sort_ind,0]<300)]
# yellow
yel_ind = v_sort_ind[(centers_hsv[v_sort_ind,1]>0.25) & (centers_hsv[v_sort_ind,0]>27) &(centers_hsv[v_sort_ind,0]<150)]
# red
red_ind = v_sort_ind[(centers_hsv[v_sort_ind,1]>0.25) &((centers_hsv[v_sort_ind,0]<27) | (centers_hsv[v_sort_ind,0]>300))]

sort_ind = np.concatenate((blue_ind[::-1], grey_ind, yel_ind, red_ind[::-1]))
print('Label sort index', sort_ind)

# visualize histogram of clusters
hist = hist_pct(cl_model.labels_)/100
bar = plot_colors(hist[sort_ind], centers[sort_ind])

# show colour chart
plt.figure(figsize=(20,2))
plt.axis("off")
plt.imshow(bar)

# generate svg plot
def hist2svg (hist, centroids):   
    print("<svg xmlns='http://www.w3.org/2000/svg' xmlns:xlink='http://www.w3.org/1999/xlink' viewBox='-5 0 1105 240' shape-rendering='CrispEdges'>")
    x_cumulative = 50
    for (percent, color) in zip(hist, centroids):
        print("<rect x='{:.2f}' y='2' width='{:.2f}' height='247' style='stroke-width: 0; stroke: none; fill: {};' />".
              format(x_cumulative, percent*1000, '#%02x%02x%02x' % tuple(np.rint(color).astype(int))))
        x_cumulative += percent*1000
    print("<rect x='45' y='5' width='1005' height='230' style='stroke-width: 10; stroke: #696969; fill: none' />")
    print("</svg>")

hist2svg(hist[sort_ind], centers[sort_ind])


In [ ]:
# RGB representation with scaling
# Mean shift

cl_model = MeanShift(bandwidth=0.2).fit(rgb)

centers = cl_model.cluster_centers_

# destandardize
centers = scaler.inverse_transform(centers) 

centers_pca = pca_model.transform(centers)

# show cluster centers with frequency labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Mean shift (bandwidth = {}) in RGB: centres and frequencies".format(cl_model.bandwidth), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

labels = [plt.text(coords[0], coords[1], '{:.1f}%'.format(label), ha='center', va='center') 
         for label, coords in zip(hist_pct(cl_model.labels_), centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)
if save_plots: plt.savefig("figs/rgb-ms-freq.svg", format="svg")


# show cluster centers with index labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Mean shift (bandwidth = {}) in RGB".format(cl_model.bandwidth), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
labels = [plt.text(coords[0], coords[1], '{}'.format(i), ha='center', va='center') 
         for i, coords in enumerate(centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)


test_clusters(centers, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw=rgb_raw)
test_clusters(centers, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw=rgb_raw)

In [ ]:
# RGB representation with scaling
# K-means

cl_model = KMeans(n_clusters=30).fit(rgb)

centers = cl_model.cluster_centers_

# destandardize
centers = scaler.inverse_transform(centers) 

centers_pca = pca_model.transform(centers)

# show cluster centers with frequency labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("K-means (n = {}) in RGB: centres and frequencies".format(cl_model.n_clusters), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

labels = [plt.text(coords[0], coords[1], '{:.1f}%'.format(label), ha='center', va='center') 
         for label, coords in zip(hist_pct(cl_model.labels_), centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)
if save_plots: plt.savefig("figs/rgb-km-freq.svg", format="svg")


# show cluster centers with index labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("K-means in RGB".format(cl_model.n_clusters), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
labels = [plt.text(coords[0], coords[1], '{}'.format(i), ha='center', va='center') 
         for i, coords in enumerate(centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)


test_clusters(centers, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw=rgb_raw)
test_clusters(centers, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw=rgb_raw)

In [ ]:
# RGB representation with scaling
# Affinity propagation clustering

cl_model = AffinityPropagation(damping=0.7, max_iter=500, convergence_iter=15).fit(rgb)

centers = cl_model.cluster_centers_

# de-standardize
centers = scaler.inverse_transform(centers)

centers_pca = pca_model.transform(centers)

fig = plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:,0], centers_pca[:,1], s=70, c=centers/255, alpha=1 )
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title('Affinity propagation (damping: {}) in RGB'.format(cl_model.damping), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
for i, coords in enumerate(centers_pca):
    plt.annotate(str(i), coords.tolist(), (coords+3).tolist())
        
test_clusters(cl_model.cluster_centers_, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw = rgb_raw)
test_clusters(cl_model.cluster_centers_, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw = rgb_raw)

In [ ]:
# HCV (XYV) representation
# set up dataset
     
hcv_raw = hsv2hcv(rgb2hsv(rgb_raw))

# convert cylindrical HCV to cartesian XYV  
hcv_xyv_raw = cyl2xyv(hcv_raw)

# standardize
scaler = StandardScaler().fit(hcv_xyv_raw)
hcv_xyv = scaler.transform(hcv_xyv_raw)

fig = plt.figure(figsize=(10,8)).add_subplot(111, projection="3d")
fig.scatter(hcv_xyv[:,0], hcv_xyv[:,1], hcv_xyv[:,2], c=rgb_raw/255, alpha = 1, marker="o", s=20)
fig.set_xlabel('C*cos(H) (z-score)')
fig.set_ylabel('C*sin(H) (z-score)')
fig.set_zlabel('V (z-score)')
if save_plots: plt.savefig("figs/hcv-3d.svg", format="svg")

In [ ]:
# create dataset for 3d web applet

#dataset = scale(hcv_xyv)

for coords, color in zip(hcv_xyv, rgb_raw):
    print("[{:.3f}, {:.3f}, {:.3f}, {:#x}],".format(coords[0], coords[1], coords[2],
                                                     int(np.dot(color, np.array([256*256, 256, 1])))))

In [ ]:
# HCV (XYV) representation
# Hierarchical clustering

#('ward', 'average', 'complete', 'single'):
cl_model = AgglomerativeClustering(linkage='ward', n_clusters=30).fit(hcv_xyv)

# assign random colour to each point according to label
cols = np.random.randint(50,200, size=(np.max(cl_model.labels_)+1, 3))[cl_model.labels_]/255
# plot
plt.figure(figsize=(9,6))
plt.scatter(rgb_raw_pca[:, 0], 
            rgb_raw_pca[:, 1], 
            c=cols, alpha = 1, marker="o", s=20)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Agg. clustering (Ward's method, n = {}) in HCV".format(cl_model.n_clusters), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
if save_plots: plt.savefig("figs/hcv-ac-all.svg", format="svg")

# calculate centers by averaging across clusters
centers = np.array([np.mean(hcv_xyv[cl_model.labels_==i], axis = 0) for i in range(np.max(cl_model.labels_)+1)])

# destandardize
centers = scaler.inverse_transform(centers) 
# hcv_xyv to rgb
centers = hsv2rgb(hcv2hsv(xyv2cyl(centers)))

centers_pca = pca_model.transform(centers)

# show cluster centers with frequency labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Agg. clustering (Ward's method, n = {}) in HCV: centres and frequencies".format(cl_model.n_clusters), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
labels = [plt.text(coords[0], coords[1], '{:.1f}%'.format(label), ha='center', va='center') 
         for label, coords in zip(hist_pct(cl_model.labels_), centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)
if save_plots: plt.savefig("figs/hcv-ac-freq.svg", format="svg")



# show cluster centers with index labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Agg. clustering (Ward's method) in HCV", 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

# add labels
labels = [plt.text(coords[0], coords[1], '{}'.format(i), ha='center', va='center') 
         for i, coords in enumerate(centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)


test_clusters(centers, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw=rgb_raw)
test_clusters(centers, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw=rgb_raw)

In [ ]:
# HCV (XYV) representation
# Mean shift clustering

cl_model = MeanShift(bandwidth = 0.5).fit(hcv_xyv)

centers = cl_model.cluster_centers_

# destandardize
centers = scaler.inverse_transform(centers) 
# hcv_xyv to rgb
centers = hsv2rgb(hcv2hsv(xyv2cyl(centers)))
#print(np.max(centers), np.min(centers))

centers_pca = pca_model.transform(centers)

# show cluster centers with frequency labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Mean shift (bandwidth = {}) in HCV: centres and frequencies".format(cl_model.bandwidth), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

labels = [plt.text(coords[0], coords[1], '{:.1f}%'.format(label), ha='center', va='center') 
         for label, coords in zip(hist_pct(cl_model.labels_), centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)
if save_plots: plt.savefig("figs/hcv-ms-freq.svg", format="svg")


# show cluster centers with index labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Mean shift in HCV", 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

# add labels
labels = [plt.text(coords[0], coords[1], '{}'.format(i), ha='center', va='center') 
         for i, coords in enumerate(centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)


test_clusters(centers, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw=rgb_raw)
test_clusters(centers, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw=rgb_raw)

In [ ]:
# HCV (XYV) representation
# K-means clustering

cl_model = KMeans(n_clusters=30).fit(hcv_xyv)

centers = cl_model.cluster_centers_

# destandardize
centers = scaler.inverse_transform(centers) 
# hcv_xyv to rgb
centers = hsv2rgb(hcv2hsv(xyv2cyl(centers)))
#print(np.max(centers), np.min(centers))

centers_pca = pca_model.transform(centers)

# show cluster centers with frequency labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("K-means (n = {}) in HCV: centres and frequencies".format(cl_model.n_clusters), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
labels = [plt.text(coords[0], coords[1], '{:.1f}%'.format(label), ha='center', va='center') 
         for label, coords in zip(hist_pct(cl_model.labels_), centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)
if save_plots: plt.savefig("figs/hcv-km-freq.svg", format="svg")


# show cluster centers with index labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("K-means in HCV", 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
labels = [plt.text(coords[0], coords[1], '{}'.format(i), ha='center', va='center') 
         for i, coords in enumerate(centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)


test_clusters(centers, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw=rgb_raw)
test_clusters(centers, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw=rgb_raw)


In [ ]:
# LAB colourspace

# set up dataset
     
lab_raw = rgb2lab(rgb_raw)

# standardize
scaler = StandardScaler().fit(lab_raw)
lab = scaler.transform(lab_raw)

fig = plt.figure(figsize=(10,8)).add_subplot(111, projection="3d")
fig.scatter(lab[:,0], lab[:,1], lab[:,2], c=rgb_raw/255, alpha = 1, marker="o", s=20)
fig.set_xlabel('L* (z-score)')
fig.set_ylabel('a* (z-score)')
fig.set_zlabel('b* (z-score)')
if save_plots: plt.savefig("figs/lab-3d.svg", format="svg")



In [ ]:
# create dataset for 3d web applet

#dataset = scale(lab)

for coords, color in zip(lab, rgb_raw):
    print("[{:.3f}, {:.3f}, {:.3f}, {:#x}],".format(coords[0], coords[1], coords[2],
                                                     int(np.dot(color, np.array([256*256, 256, 1])))))

In [ ]:
# Lab representation with scaling
# Hierarchical clustering

#('ward', 'average', 'complete', 'single'):
cl_model = AgglomerativeClustering(linkage='ward', n_clusters=30).fit(lab)

# assign random colour to each point according to label
cols = np.random.randint(50,200, size=(cl_model.n_clusters, 3))[cl_model.labels_]/255
# plot
plt.figure(figsize=(9,6))
plt.scatter(rgb_raw_pca[:, 0], 
            rgb_raw_pca[:, 1], 
            c=cols, alpha = 1, marker="o", s=20)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Agg. clustering (Ward's method, n = {}) in Lab".format(cl_model.n_clusters), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
if save_plots: plt.savefig("figs/lab-ac-all.svg", format="svg")

# calculate centers by averaging across clusters
centers = np.array([np.mean(lab[cl_model.labels_==i], axis = 0) for i in range(cl_model.n_clusters)])

# destandardize
centers = scaler.inverse_transform(centers) 

# convert to rgb
centers = lab2rgb(centers)
centers_pca = pca_model.transform(centers)

# show cluster centers with frequency labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Agg. clustering (Ward's method, n = {}) in Lab: centres and frequencies".format(cl_model.n_clusters), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

labels = [plt.text(coords[0], coords[1], '{:.1f}%'.format(label), ha='center', va='center') 
         for label, coords in zip(hist_pct(cl_model.labels_), centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)
if save_plots: plt.savefig("figs/lab-ac-freq.svg", format="svg")


# show cluster centers with index labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Agg. clustering (Ward's method) in Lab", 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

# add labels
labels = [plt.text(coords[0], coords[1], '{}'.format(i), ha='center', va='center') 
         for i, coords in enumerate(centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)


test_clusters(centers, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw=rgb_raw)
test_clusters(centers, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw=rgb_raw)

In [ ]:
# Lab representation
# Mean shift clustering

cl_model = MeanShift(bandwidth = 0.5).fit(lab)

centers = cl_model.cluster_centers_

# destandardize
centers = scaler.inverse_transform(centers) 
# hcv_xyv to rgb
centers = lab2rgb(centers)
#print(np.max(centers), np.min(centers))

centers_pca = pca_model.transform(centers)

# show cluster centers with frequency labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Mean shift (bandwidth = {}) in Lab: centres and frequencies".format(cl_model.bandwidth), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

labels = [plt.text(coords[0], coords[1], '{:.1f}%'.format(label), ha='center', va='center') 
         for label, coords in zip(hist_pct(cl_model.labels_), centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)
if save_plots: plt.savefig("figs/lab-ms-freq.svg", format="svg")


# show cluster centers with index labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("Mean shift in Lab", 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
labels = [plt.text(coords[0], coords[1], '{}'.format(i), ha='center', va='center') 
         for i, coords in enumerate(centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)


test_clusters(centers, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw=rgb_raw)
test_clusters(centers, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw=rgb_raw)

In [ ]:
# Lab representation
# K-means clustering

cl_model = KMeans(n_clusters=30).fit(lab)

centers = cl_model.cluster_centers_

# destandardize
centers = scaler.inverse_transform(centers) 
# hcv_xyv to rgb
centers = lab2rgb(centers)
#print(np.max(centers), np.min(centers))

centers_pca = pca_model.transform(centers)

# show cluster centers with frequency labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("K-means (n = {}) in Lab: centres and frequencies".format(cl_model.n_clusters), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

labels = [plt.text(coords[0], coords[1], '{:.1f}%'.format(label), ha='center', va='center') 
         for label, coords in zip(hist_pct(cl_model.labels_), centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)
if save_plots: plt.savefig("figs/lab-km-freq.svg", format="svg")


# show cluster centers with index labels
plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:, 0], 
            centers_pca[:, 1], 
            c=centers/255, alpha = 1, marker="o", s=100)
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title("K-means in Lab", 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})

# add labels
labels = [plt.text(coords[0], coords[1], '{}'.format(i), ha='center', va='center') 
         for i, coords in enumerate(centers_pca)]
adjust_text(labels, expand_points = (1.3, 1.4)) # arrowprops=dict(arrowstyle="-", color='black', lw=0.2)


test_clusters(centers, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw=rgb_raw)
test_clusters(centers, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw=rgb_raw)


In [ ]:
# Lab representation with scaling
# Affinity propagation clustering

cl_model = AffinityPropagation(damping=0.7, max_iter=500, convergence_iter=15).fit(lab)

centers = cl_model.cluster_centers_

# de-standardize
centers = scaler.inverse_transform(centers)
# lab to rgb
centers = lab2rgb(centers)

centers_pca = pca_model.transform(centers)

fig = plt.figure(figsize=(9,6))
plt.scatter(centers_pca[:,0], centers_pca[:,1], s=70, c=centers/255, alpha=1 )
plt.xlabel(str(np.round(pca_model.components_[0], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.ylabel(str(np.round(pca_model.components_[1], 2))+'([R G B] - '+str(np.round(np.mean(rgb_raw, axis=0), 0))+')' )
plt.title('Affinity propagation (damping = {}) in Lab'.format(cl_model.damping), 
          fontdict={'fontsize': 14,
                    'fontweight' : 14,
                    'verticalalignment': 'baseline',
                    'horizontalalignment': 'center'})
for i, coords in enumerate(centers_pca):
    plt.annotate(str(i), coords.tolist(), (coords+3).tolist())
        
test_clusters(cl_model.cluster_centers_, cl_model.labels_, lower = (0,0,0), upper = (125,125,125), raw = rgb_raw)
test_clusters(cl_model.cluster_centers_, cl_model.labels_, lower=(240,240,240), upper=(255,255,255), raw = rgb_raw)